In [1]:
!pip install pdfminer
!pip install pypdf2

In [2]:
import re
from re import search
import os
from sklearn.preprocessing import scale

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import HTMLConverter
from io import StringIO
from bs4 import BeautifulSoup
import PyPDF2
import xlwt
#from main import scanner
import pandas as pd
import numpy as np

In [3]:
def conversion_html(path):
    Manager = PDFResourceManager()
    string_io = StringIO()
    laparams = LAParams()
    device = HTMLConverter(Manager, string_io, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(Manager, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    fp.close()
    device.close()
    text = string_io.getvalue()
    return text

In [4]:
def data_manipulation(html, res):
    soup = BeautifulSoup(html, 'html.parser')
    if res == 0:
        return soup.get_text()
    pix = re.compile("font-size:(\d+)")
    if res == 1:
        empty = []
        font_size = {}
        for tag in soup.select("[style*=font-size]"):
            font_size[tag.text.strip()] = pix.search(tag["style"]).group(1)
        for cle in font_size.keys():
            if str(cle) == '':
                empty.append(cle)
        for i in empty:
            del font_size[i]
        return font_size
    if res == 2:
        empty = []
        font_family = {}
        pix = re.compile("font-family:(\s\w+)([\s,+\w+,-]*)")
        for tag in soup.select("[style*=font-family]"):
            font_family[tag.text.strip()] = pix.search(tag["style"]).group()
        for cle in font_family.keys():
            if str(cle) == '':
                empty.append(cle)
        for i in empty:
            del font_family[i]
        return font_family

In [5]:
def data_manipulation(html, res):
    soup = BeautifulSoup(html, 'html.parser')
    if res == 0:
        return soup.get_text()
    pix = re.compile("font-size:(\d+)")
    if res == 1:
        empty = []
        font_size = {}
        for tag in soup.select("[style*=font-size]"):
            font_size[tag.text.strip()] = pix.search(tag["style"]).group(1)
        for cle in font_size.keys():
            if str(cle) == '':
                empty.append(cle)
        for i in empty:
            del font_size[i]
        return font_size
    if res == 2:
        empty = []
        font_family = {}
        pix = re.compile("font-family:(\s\w+)([\s,+\w+,-]*)")
        for tag in soup.select("[style*=font-family]"):
            font_family[tag.text.strip()] = pix.search(tag["style"]).group()
        for cle in font_family.keys():
            if str(cle) == '':
                empty.append(cle)
        for i in empty:
            del font_family[i]
        return font_family


In [6]:
def data_cleaning(path,i):
    dict = data_manipulation(conversion_html(path), i)
    pop_list = []
    dict1 = {}
    dict_n = []
    for cle in dict.keys():
        cle=str(cle).replace('\n','  ')
        if len(str(cle)) <= 3:
            pop_list.append(cle)
    for i in range(0, len(pop_list)):
        dict.pop(pop_list[i])
    for cle1 in dict.keys():
        cle1 = re.sub('[^A-Za-z0-9 ' ' | éèà À Á Â Æ Ç È É Ê Ë Ì Í Î Ï Ñ Ò Ó Ô Œ Ù Ú Û Ü Ý Ÿà á â æ ç è é ê'
                      '	ë ì	í î	ï ñ	ò ó	ô œ	ù ú	û ü	ý ÿ]+', '', str(cle1))
        cle1 = re.sub('[" "][" "]+','  ',str(cle1))
        dict_n.append(cle1)
    s = 0
    for value in dict.values():
        dict1[dict_n[s]] = value
        s=s+1
    pop_list = []
    for cle in dict1.keys():
        if str(cle).startswith(','):
            cle = str(cle).replace(',', ' ', 1)
        if len(str(cle)) <= 3:
            pop_list.append(cle)
    for i in range(0, len(pop_list)):
        dict1.pop(pop_list[i])

    return dict1


In [ ]:
conda install -c conda-forge python-levenshtein

In [7]:
import nltk
from Levenshtein import distance

In [8]:
def nb_mots(chaine):
    liste = chaine.split()
    l = len(liste)
    return l


def case(chaine):
    if chaine.isupper():
        return 1
    else:
        return 0


def font_size(path):
    font_size = data_cleaning(path, 1)
    s = 0
    cles = 0
    for value in font_size.values():
        s = s + float(value)
        cles = cles + 1
    average = s / cles
    for cle in font_size.keys():
        if float(font_size[cle]) < average:
            font_size[cle] = 0
        else:
            font_size[cle] = 1
    return font_size


def font_family(path):
    i = 0
    cles = 0
    s = 0
    font_family = data_cleaning(path, 2)
    fonts = {}
    for cle in font_family.keys():
        for cle1 in font_family.keys():
            if font_family[cle] == font_family[cle1]:
                i = i + 1
        fonts[font_family[cle]] = i
        i = 0
    for cle in fonts.keys():
        cles = cles + 1
        s = s + fonts[cle]
    average = s / cles
    for cle in fonts.keys():
        if float(fonts[cle]) < average:
            fonts[cle] = 0
        else:
            fonts[cle] = 1
    for cle in font_family.keys():
        font_family[cle] = fonts[font_family[cle]]
    return (font_family)


def bold_exist(path):
    font_family = data_cleaning(path, 2)
    for cle in font_family.keys():
        if "Bold" in font_family[cle] or "bold" in font_family[cle]:
            font_family[cle]=1
        else:
            font_family[cle]=0
    return font_family

def nb_noms(chaine):
    s = 0
    tab = nltk.pos_tag(nltk.word_tokenize(chaine))
    for i in range(0, len(tab)):
        fonc = tab[i][1]
        if fonc.startswith('NN'):
            s = s + 1
    return s


def nb_VERBS(chaine):
    s = 0
    tab = nltk.pos_tag(nltk.word_tokenize(chaine))
    for i in range(0, len(tab)):
        fonc = tab[i][1]
        if fonc.startswith('VB'):
            s = s + 1
    return s


def cmp(chaine1, chaine2):
    chaine2 = chaine2.replace('/n', '')
    edit_dist = distance(chaine1, chaine2)
    return edit_dist


In [9]:
def to_excel(max):
    workbook = xlwt.Workbook()
    for j in range(1,max+1):
        sheet = workbook.add_sheet(f'{j}', cell_overwrite_ok=True)
        f = 0
        list = ['bloc', 'label', 'font_size', 'font_family', 'nb_mots', 'nb_verbes', 'nb_noms', 'case', 'bold']
        for i in range(0, 9):
            sheet.write(0, f, list[i])
            f = f + 1
        dict = data_cleaning(f'dataset_resume/data/{j}.pdf',2)
        size = font_size(f'dataset_resume/data/{j}.pdf')
        family = font_family(f'dataset_resume/data/{j}.pdf')
        bold = bold_exist(f'dataset_resume/data/{j}.pdf')
        i = 1
        for cle in dict.keys():
            sheet.write(i, 0, cle)
            sheet.write(i,4,nb_mots(cle))
            sheet.write(i,5,nb_VERBS(cle))
            sheet.write(i,6,nb_noms(cle))
            sheet.write(i,7,case(cle))




            i = i + 1
        i=1
        for cle in size.keys():
            sheet.write(i,2,size[cle])
            i=i+1
        i=1
        for cle in family.keys():
            sheet.write(i,3,family[cle])
            i=i+1

        i = 1
        for cle in bold.keys():
            sheet.write(i, 8, bold[cle])
            i = i + 1

    workbook.save('dataset_resume/resume.xls')


In [10]:
def input_cv(path,id):
  workbook = xlwt.Workbook()
  sheet = workbook.add_sheet('1', cell_overwrite_ok=True)
  f = 0
  list = ['bloc', 'label', 'font_size', 'font_family', 'nb_mots', 'nb_verbes', 'nb_noms', 'case', 'bold']
  for i in range(0, 9):
      sheet.write(0, f, list[i])
      f = f + 1

  dict = data_manipulation(conversion_html(path),2)
  size = font_size(path)
  family = font_family(path)
  bold = bold_exist(path)
  i = 1
  for cle in dict.keys():
      sheet.write(i, 0, cle)
      sheet.write(i,4,nb_mots(cle))
      sheet.write(i,5,nb_VERBS(cle))
      sheet.write(i,6,nb_noms(cle))
      sheet.write(i,7,case(cle))
      i = i + 1
  i = 1
  for cle in size.keys():
      sheet.write(i, 2, size[cle])
      i = i + 1
  i = 1
  for cle in family.keys():
      sheet.write(i, 3, family[cle])
      i = i + 1

  i = 1
  for cle in bold.keys():
      sheet.write(i, 8, bold[cle])
      i = i + 1
  workbook.save(f'Uses/Resume/{id}.xls')

  read_file = pd.read_excel(f'Uses/Resume/{id}.xls' ,index_col=None)
  read_file.to_csv(f'Uses/Resume/{id}.csv', index=None, header=True)
  df = pd.read_csv(f'Uses/Resume/{id}.csv', usecols=[j for j in range(1, 9)])
  mat = df.to_numpy()
  np.savetxt(f'Uses/Resume/{id}{id}.csv', mat, delimiter=",")
  os.remove(f'Uses/Resume/{id}.csv')

  



In [11]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [12]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
input_cv('Uses/Resume/aa.pdf',3)

In [15]:
def segmentation(segment):
  read_file = pd.read_excel('Uses/Resume/segmentation.xls' ,index_col=None)
  read_file.to_csv('Uses/Resume/segmentation.csv', index=None, header=True)
  df1=pd.read_csv("Uses/Resume/segmentation.csv",header=None)
  liste = []
  for i in range(0,len(df1[segment])):
    if(str(df1[segment][i])) != 'nan':
      liste.append(df1[segment][i])
  return (liste)

objective = segmentation(0)
work = segmentation(1)
education = segmentation(2)
hard = segmentation(3)
soft = segmentation(4)
achievement = segmentation(5)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()


In [17]:
def similarite(seg_cv,segment):
  
  res = []
  for i in range (0,len(segment)):
    X = vectorizer.fit_transform([segment[i],seg_cv])
    tfidf = ((X*X.T).A)[0,1]
    res.append(tfidf)
  return(max(res))

In [18]:
def bloc_recognition(seg_cv):
  obj = similarite(seg_cv,objective)
  w = similarite(seg_cv,work)
  ed = similarite(seg_cv,education)
  h = similarite(seg_cv,hard)
  so = similarite(seg_cv,soft)
  ach = similarite(seg_cv,achievement)
  f = (max(obj,w,ed,h,so,ach))
  if(obj==f and f>0.5):
    return('Profil')
  if(w==f and f>0.5):
    return('Experience professionelle')
  if(ed==f and f>0.5):
    return('Expérience académique')
  if(h==f and f>0.5):
    return('Hard skills et projets')
  if(so==f and f>0.5):
    return('Soft skills')
  if(ach==f and f>0.5):
    return('certifications et compétitions')
  else:
    return(' ')



In [19]:
def segment(id_candidat,path):
  from tensorflow.keras.models import load_model
  saved_model = load_model('Uses/models/model13.h5')
  input_cv(path,id_candidat)
  df1=pd.read_csv(f'Uses/Resume/{id_candidat}{id_candidat}.csv',header=None)
  x1 = df1.drop(0,axis=1)
  x_pred1 = scale(x1)
  predictions1 = saved_model.predict(x_pred1)
  p1 =np.argmax(predictions1, axis=1)
  
  vectorizer = TfidfVectorizer()
  dict_cv= data_manipulation(conversion_html(path),2)
  d = dict_cv
  text = []
  for cle in d:
    text.append(cle)
  
  headlines = []
  for i in range(0,len(text)):
    if(p1[i]==1):
      headlines.append(text[i])
  
  liste1 = []
  for i in range(0,len(headlines)):
    a = bloc_recognition(headlines[i])
    liste1.append(a)
  for cle in d:
    if (cle not in headlines):
      if((similarite(cle,work))>0.5):
        headlines.append(cle)
        liste1.append('Experience professionelle')

      if((similarite(cle,objective))>0.5):
        headlines.append(cle)
        liste1.append('Profil')
        
      if similarite(str(cle),education)> 0.5:
        headlines.append(cle)
        liste1.append('Expérience académique')

      if(similarite(cle,hard)>0.5):
        headlines.append(cle)
        liste1.append('Hard skills et projets')

      if(similarite(cle,soft)>0.5):
        headlines.append(cle)
        liste1.append('Soft skills')

      if(similarite(cle,achievement)>0.5):
        headlines.append(cle)
        liste1.append('certifications et compétitions')
    segments = []
  for cle in d:
    if (cle not in headlines ):
      d[cle] = ' '
    else:
      index = headlines.index(cle)
      d[cle] = liste1[index]
    seg = {}
  for cle in list(d):
    if (d[cle]==' ') :
      del d[cle]
    else:
      break
  z = list(d)
  for i in range(0,len(z)):
    if(d[z[i]]!=' '):
      seg[z[i]] = d[z[i]]
      e = i+1
      for j in range(e,len(z)):
        if(d[z[j]]==' '):
          seg[z[j]] =d[z[i]]
            

  for cle in list(seg):
    if(cle in headlines):
      del seg[cle]
    
  segment ={}
  for cle in seg:
    segment[seg[cle]] = ' '
    
  for cle in seg:
    segment[seg[cle]] = segment[seg[cle]] + '\n' +  cle
  return(segment)
    
 
 
 


      
  

  

  






In [22]:
segment = segment(5,'Uses/Resume/aa.pdf')
for cle in segment:
  print(f'section : {cle}')
  print(segment[cle])
  print('----------------------------')

section : Hard skills et projets
 
JAVA
C
C++
PYTHON
HTML
CSS
JavaScript
PHP
MySQL
Oracle
LINUX
NETLOGO
Hibernate
Swing/AWT
CodeBlocks
Eclipse

Arabe (Langue  Maternelle)
Français  (Bilingüe)
Anglais    (Professionnel)
Allemand  (Notions  Scolaires)
Une Simulation Robotique pour l'Expérimentation des Méthodes 
d'Apprentissage par Renforcement
Février
–
Mai 2021
Il s’agit d’expérimenter  plusieurs méthodes d'apprentissage par
renforcement en vue d'une étude comparative de celles-ci.
Key words
:
PROJET D’ETE
Août - sept. 2020
Cette application  permet à tous les étudiants de l'ENSI de récupérer un 
formulaire ou autre numériquement.
JAV A, SWING, AWT, MySQL
Jeu « Mots Meles »
Avril
juin 2020
Il s’agit de retrouver l’ensemble des mots proposés dans la liste à côté
de la grille.
----------------------------
section : Soft skills
 
Sport
Cinéma
Arij SOULA
Élève   ingénieure  en 2
ème
Année  Sciences de
l’Informatique,
A la recherche d’un stage d’été de 2 mois
----------------------------
s